Welcome to week 9, of the [Noisebridge Python Class](https://github.com/audiodude/PythonClass), part 2 of the Web Apps lesson!

In this lesson you will learn:

* Setting up an SQL schema for use with the app (repeat of SQL lesson)
* Receiving input from the URL and URL matching
* Receiving input using GET/query parameters
* Receiving user input from a `<form>` and creating a new row in a database
* Using templating to display a list of the data added to the app

Okay, let's go!

Similar to what we did in [Lesson 5](https://sfpythonlab.com/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Faudiodude%2FPythonClass&branch=main&urlpath=tree%2FPythonClass%2Flessons%2F05-sql%2Fsql_in_python.ipynb), we can set up a schema for our web app, in order to store and persist data.

We will use the same schema we used there. As a reminder it's:

```sql
CREATE TABLE IF NOT EXISTS links (
  id INTEGER NOT NULL PRIMARY KEY,
  name VARCHAR(255),
  url TEXT,
  created_at TIMESTAMP,
  upvotes INTEGER DEFAULT 0,
  downvotes INTEGER DEFAULT 0
);

CREATE TABLE IF NOT EXISTS users (
  id INTEGER NOT NULL PRIMARY KEY,
  email TEXT,
  hashed_password VARCHAR(255),
  profile TEXT
);
```

This data is available in `schema.sql` in this directory.

To load the schema in a sqlite database, we can use the command line to do that process "offline", aka outside of the code of the web app itself. From the `radish/` directory we run:


```
sqlite3 db.sqlite < ../schema.sql
```

This runs all of the SQL commands in schema.sql as input to the `sqlite3 db.sqlite` comand, which then creates the given database from those commands (which are database creation commands).

We can take user input directly from the URL using "URL matching". Let's look at an example (in `01-url-matching`):

In [ ]:
import flask

app = flask.Flask(__name__)

@app.route('/hello/<name>')
def hello(name):
  return f'Hello {name}'

If we go to http://localhost:5000/hello/Abby, we see "Hello Abby". Note that the name of the function doesn't matter, but the paramter name needs to match the value in angle brackets in the route (`name`).

*If you haven't read through lesson 8, web apps part 1, yet, now would be a good time to read about setting up an activating a virtual env, and installing and running Flask.*

We can use multiple routes on the same function along with a default value to provide a greeting even when the name is omitted:

In [ ]:
import flask

app = flask.Flask(__name__)

@app.route('/hello/')
@app.route('/hello/<name>')
def hello(name='World'):
  return f'Hello {name}'

*(This is the example in 01-url-matching/main.py)*

We have to provide a default parameter to `name`, making it a keyword argument, so that if the first route is triggered and it doesn't have a value, the function can still run.

We can also use what are known as **query parameters** to capture the value we wish to display. You've probably seen query parameters (maybe sometimes called "GET parameters") in URLs before, like:

https://www.google.com/search?q=flask+python&rlz=1C1CHBF_enUS1029US1029&

Where everything after the `?` is the query. Query parameters come as key/value pairs, where the key is on the left side of the equal sign and the value is on the right. So in the above, `q=flask+python` means that the key `q` has a value of `flask+python` (+ signs in query parameters represent spaces, and will be sent to your web app as such).

Let's re-write our flask app to use query parameters:

In [3]:
import flask

app = flask.Flask(__name__)

@app.route('/hello/')
def hello():
  name = flask.request.args.get('name', 'World')
  return f'Hello {name}'

Here we use the special flask variable `request`, which contains lots of useful information about the incoming request. We're interested in `args`, which operates almost like a Python dictionary with a few special exceptions. We use the `get` method of args to retrieve the query with the key `name`, defaulting to the string `'World'` if it doesn't exist.

When we visit http://localhost:5000/hello/?name=Abby we see "Hello Abby" and when we visit http://localhost:5000/hello we see "Hello World". One thing to note about query params is that there is no inherent error if they are missing or if additional parameters are specified. We can visit http://localhost:5000/hello/?naame=Abby&foo=bar and see that we get "Hello World" (since name is mispelled: `naame`, there is no entry for `name` in the params).

Now, most web apps don't expect the users to put parameters directly in the URL. We should create a form for greeting people. Here is `templates/index.html`:

```html
<!DOCTYPE html>
<html>
  <head>
    <title>Hello</title>
  </head>
  <body>
    <form action="/hello">
      Name: <input name="name" placeholder="Your name" /><br />
      <button>Enter</button>
    </form>
  </body>
</html>
```

When we add a handler for the main page of the site, we can visit it and see this form:

![Screenshot of a simple web form](https://pixelfed.social/storage/m/_v2/588554065884192073/c51ea9690-856390/XLASpCaktHQy/iWmmCS2KcWpUh9qmyUzL1YgFwMVLx6eM2Fw3OWQj.png)

When we fill out the form and click "Enter", the browser converts the parameters to query parameters and sends us to `/hello/`, which we specified as the `action` of the `<form>`.

*(The code for this part is in 02-get-params)*

---

Query parameters are useful when you want to capture data from the user, but you want any visitor to the URL to see the page rendered with those params. For example, my link to Google above will work if you click on it, and will take you to a page with the same query as the one I entered when I copied the URL. In the same way, going to http://localhost:5000/hello?name=Brad will always show "Hello Brad".

However, sometimes we want to use input from the user to alter data in our application. If we want to build the *Radish* application, we will need some way to input the URL data. We *could* use query parameters for that, and we might end up with a URL like:

http://localhost:5000/link/new?name=Google&url=http://google.com&...

The problem with this approach is that if a user shares a link to this URL, we will create a new "link" entry in our database every time it is accessed. For requests that mutate data, we should use **POST parameters**.

 While a GET request is made every time you type a URL, for POST parameters the browser must make a special type of request to the web app called a **POST request**.

To capture and process POST parameters, we make a couple small tweaks to our web app. First, we change the HTML to add `method="POST"` to the form, to tell the browser to make the request using a POST action instead of GET with query parameters. As part of this, the parameters are transferred "invisibly" as part of the request body (instead of being in the URL). Rest assured, they are being transferred.

```html
<!DOCTYPE html>
<html>
  <head>
    <title>Hello</title>
  </head>
  <body>
    <form action="/hello" method="POST">
      Name: <input name="name" placeholder="Your name" /><br />
      <button>Enter</button>
    </form>
  </body>
</html>
```

Next we change our application functions. We specify in our route that the `hello` function now should be accessed using only POST requests. We also use `flask.request.form` instead of `flask.request.args`. The former is for POST and the latter is for GET (a simple Google search will help you if you ever forget this distinction).

In [5]:
import flask

app = flask.Flask(__name__)

@app.route('/')
def index():
  return flask.render_template('index.html')
  
@app.route('/hello/', methods=["POST"])
def hello():
  name = flask.request.form.get('name', 'World')
  return f'Hello {name}'

---

Now that we know how to process POST data, we can create a form for users to add links to Radish:

```html
<!DOCTYPE html>
<html>
  <head>
    <title>Radish - Add a link</title>
  </head>
  <body>
    <form action="/links/create" method="POST">
      <div>URL: <input name="url" placeholder="Site URL" /></div>
      <div>Name: <input name="name" placeholder="Name of site" /></div>
      <button>Create Link</button>
    </form>
  </body>
</html>
```

Now we need a function in our web app to process data from this form and add the data to the database:

In [6]:
from contextlib import closing
import sqlite3

import flask

app = flask.Flask(__name__)

@app.route('/')
def index():
  return flask.render_template('index.html')

@app.route('/links/new')
def new_link():
  return flask.render_template('new_link.html')
  
@app.route('/links/create', methods=["POST"])
def create_link():
  name = flask.request.form.get('name')
  url = flask.request.form.get('url')
  if not name or not url:
    flask.abort(400)

  db = sqlite3.connect('../db.sqlite')
  with closing(db.cursor()) as cursor:
    db.execute('''
      INSERT INTO links (name, url) VALUES (?, ?)
    ''', (name, url))
  db.commit()

  return flask.redirect('/')

While we could have had the same function render `/links/new` and `/links/create`, it would have meant making that function accessible by both GET and POST. We would then have to inspect which method was used and either display the form or process the data. It's cleaner to have them live at separate URLs.